# tsam - Optimal combination of segments and periods 
Determine the optimal combination of segments and periods during the aggregation of the time series
Date: 27.04.2022

Author: Leander Kotzur

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

import tsam
import tsam.hyperparametertuning as tune
from tsam.timeseriesaggregation import TimeSeriesAggregation

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)
raw = raw.rename(
    columns={
        "T": "Temperature [°C]",
        "Load": "Load [kW]",
        "Wind": "Wind [m/s]",
        "GHI": "Solar [W/m²]",
    }
)

Use tsam's built-in heatmap plotting for visual comparison of the time series

Plot an example series - in this case the temperature

In [ ]:
# Original data heatmaps
tsam.plot.heatmaps(raw, period_hours=24, title="Original Data")

### Tune a hierarchical aggregation with segments in combination with duration representation

In [ ]:
tunedAggregations = tune.HyperTunedAggregations(
    TimeSeriesAggregation(
        raw,
        hoursPerPeriod=24,
        clusterMethod="hierarchical",
        representationMethod="durationRepresentation",
        distributionPeriodWise=False,
        rescaleClusterPeriods=False,
        segmentation=True,
    )
)

And determine the pareto optimal aggregation up to 200 total time steps. This may take some time...

In [ ]:
tunedAggregations.identifyParetoOptimalAggregation(untilTotalTimeSteps=8760)

And show the results for the last aggregation

In [ ]:
predictedPeriods = tunedAggregations.aggregationHistory[-1].predictOriginalData()

In [ ]:
# Reconstructed data heatmaps
tsam.plot.heatmaps(
    predictedPeriods,
    reference_data=raw,
    period_hours=tunedAggregations.base_aggregation.hoursPerPeriod,
    title="Reconstructed Data",
)

Save the resulting combination

In [ ]:
pd.DataFrame(
    {
        "segments": tunedAggregations._segmentHistory,
        "periods": tunedAggregations._periodHistory,
    }
).to_csv(os.path.join("results", "paretoOptimalAggregation.csv"))